In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import requests
print(torch.cuda.is_available())
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_default_device(device) 

True


In [9]:
class MaligaroDataSet(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]
    
    @property
    def classes(self):
        return self.data.classes
    
df = yf.Ticker("AAPL").history(period="max")
e = torch.tensor(np.array(df))
e.device

device(type='cuda', index=0)

In [10]:
# Define a simple neural network
class StockPriceModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(StockPriceModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# df = yf.Ticker("AAPL").history(period="max")
# Prepare the data
df['Return'] = df['Close'].pct_change()
df = df.dropna()
X = df[['Open', 'High', 'Low', 'Close', 'Volume']].values
y = df['Return'].values

# Convert data to tensors
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1).to(device)

# Create dataset and dataloader
dataset = MaligaroDataSet(list(zip(X_tensor, y_tensor)))
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Initialize the model, loss function, and optimizer
input_size = X.shape[1]
hidden_size = 128
output_size = 1
model = StockPriceModel(input_size, hidden_size, output_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

RuntimeError: Expected a 'cuda' device type for generator but found 'cpu'